In [1]:
import collections
import copy
import datetime
import itertools
import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
G = cibblbibbl.CIBBL
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay

In [9]:
data = collections.defaultdict(list)
for Mu in G.matchups:
    for teamId, d0 in Mu.config["player"].items():
        for playerId, p in d0.items():
            Pl = player(str(playerId))
            prespp = Pl.prespp(Mu)
            spp = p.get("spp", 0)
            data[Pl].append((Mu, prespp, spp))

In [10]:
uncommon = collections.defaultdict(list)
for Pl in sorted(data):
    L = data[Pl]
    if not isinstance(Pl, (cibblbibbl.player.Player, cibblbibbl.player.RaisedDeadPlayer)):
        continue
    expprespp = 0
    if Pl.prev:
        if data.get(Pl.prev):
            prevlastd = data[Pl.prev][-1]
        else:
            prevlastd = [None, 0, 0]
        expprespp = sum(prevlastd[1:3]) * Pl.prevsppmul
    nameprint = False
    for i, (Mu, prespp, spp) in enumerate(L):
        if prespp != expprespp:
            if not nameprint:
                s = f'[{Pl.Id}, {Pl.status}] {Pl.name} [{Pl.team.Id}, {Pl.team.name}]'
                if Pl.prev:
                    s += f' (prev: [{Pl.prev.Id}] {Pl.prev.name})'
                print(s)
                nameprint = True
            print(f'{Mu.match.Id}, exp:{expprespp}, got:{prespp}')
            uncommon[Pl].append([Mu, expprespp, prespp])
        expprespp = prespp + spp       
    if nameprint:
        print()

In [1]:
# carry previous SPPs (reload above two after it runned)
for Pl, L in uncommon.items():
    Pl0 = Pl.prev
    if not Pl0:
        continue
    prevmatchups = tuple(
        cibblbibbl.matchup.sort_by_modified(
            Mu
            for S in Pl0._matchups.values()
            for Mu in S
        )
    )
    if not prevmatchups:
        continue
    prevspp = prevmatchups[-1].performance(Pl0).get("prespp", 0)
    if prevspp:
        adminspp = Pl.adminspp
        adminspp["0"] = prevspp
        Pl.adminspp = adminspp
    print(Pl.name, prevspp)

    

NameError: name 'uncommon' is not defined

In [8]:
#fix admin spps
for Pl, L in uncommon.items():
    for Mu, expprespp, prespp in L:
        Ma = Mu.match
        value = expprespp - prespp
        adminspp = Pl.adminspp
        adminspp[str(Ma.Id)] = value
        print(Pl.name, Ma.Id, value)

Kkrozh 4046041 -3
Zzak 4046041 -14
Tarh 4046041 -14
Gashokh 4055462 -4
Vanderburth 4055462 -4


In [5]:
Pl = player(12138861)
uncommon[Pl]

[[Matchup('cibbl', '44387', 3, 882382, 896134), 32, 0]]

In [6]:
Mu = Pl.matchups("cibbl")[0]

In [21]:
Pl.prespp(Mu)

32

In [8]:
Pl0 = Pl.prev

In [9]:
prevmatchups = tuple(
    cibblbibbl.matchup.sort_by_modified(
        Mu
        for S in Pl0._matchups.values()
        for Mu in S
    )
)

In [10]:
if prevmatchups:
    prevspp = prevmatchups[-1].performance(Pl0).get("prespp", 0)
else:
    prevspp = 0
prevspp

32

In [19]:
if prevspp:
    adminspp = Pl.adminspp
    adminspp["0"] = prevspp
    Pl.adminspp = adminspp

NormalPlayer('12138861') adminspp {'0': 32}


In [17]:
Pl.adminspp

{'0': 32}

In [18]:
Pl.config["adminspp"] = adminspp

In [14]:
{'0': 32} == {}

False

In [24]:
default = Pl.__class__.adminspp.default(Pl)

In [30]:
a = Pl.__class__.adminspp.__get__(Pl, Pl.__class__)
a

{'0': 32}

In [31]:
a == default

True

In [32]:
a is default

False

In [34]:
default

{}

In [35]:
type(a)

cibblbibbl.jsonfile.JSONFileObject

In [36]:
type(default)

dict

In [39]:
a== default

True

In [41]:
hash(a)

TypeError: unhashable type: 'JSONFileObject'